In [1]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time
import pandas as pd
import numpy as np
import pyodbc
import logging
import traceback, sys
import smtplib
from email.mime.text import MIMEText
import socket

In [2]:
# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP'}
user142 = 'cmapp'
pwd142 = '0000'
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBdata['ip'] + \
              ';DATABASE=' + DBdata['db'] + ';UID=' + user142 + ';PWD=' + pwd142)
DBdata_cr = conn_DBdata.cursor()
TableName = 'FB_AdsData'

## 連接SQL新寫法

In [3]:
# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP', 'user142': 'cmapp', 'pwd142': '0000'}
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};'\
                            'SERVER=' + DBdata['ip'] + \
                            ';DATABASE=' + DBdata['db'] +\
                            ';UID=' + DBdata['user142'] +\
                            ';PWD=' + DBdata['pwd142'] + ';')

DBdata_cr = conn_DBdata.cursor()
TableName = 'FB_AdsData'

## 從SQL製作全部的DF

In [ ]:
"""
        SELECT TOP(100) [Serial_Number]
      ,[Campaign_Name]
      ,[Adset_Name]
      ,[Ad_Name]
      ,[Objective]
      ,[Amount_Spent]
      ,[Amount_Install]
      ,[Amount_Purchase]
      ,[Purchase_Conversion_Value]
      ,[Amount_Subscribe]
      ,[Subscribe_Conversion_Value]
      ,[CPM]
      ,[CTR]
      ,[Impressions]
      ,[Reporting_Date]
      ,[Category]
        FROM [CMAPP].[dbo].[{}]
        """

In [13]:
def Get_SN_RD(TableName):
    get_both = """
        SELECT TOP(10)*
        FROM [CMAPP].[dbo].[{}]
        """
    print(get_both.format(TableName))
    DBdata_cr.execute(get_both.format(TableName))
    print([x[0] for x in DBdata_cr.description])
    print([row for row in DBdata_cr.fetchall()])
    
#     df = pd.DataFrame(DBdata_cr.fetchall())
#     df.columns = [x[0] for x in DBdata_cr.description]
    conn_DBdata.commit()
    return df

df = Get_SN_RD(TableName)
df


        SELECT TOP(10)*
        FROM [CMAPP].[dbo].[FB_AdsData]
        
['Serial_Number', 'Campaign_Name', 'Adset_Name', 'Ad_Name', 'Objective', 'Amount_Spent', 'Amount_Install', 'Amount_Purchase', 'Purchase_Conversion_Value', 'Amount_Subscribe', 'Subscribe_Conversion_Value', 'CPM', 'CTR', 'Impressions', 'Reporting_Date', 'Category']
[(Decimal('202001010000'), '期貨電子盤_下載_IOS(舊)', '興趣期貨受眾_IOS', '創意廣告', 'APP_INSTALLS', Decimal('139'), Decimal('7'), Decimal('1'), Decimal('8888'), None, None, Decimal('57.17812'), Decimal('1.19293'), Decimal('2431'), '20200101', 'FB'), (Decimal('202001010001'), '期貨電子盤_下載_IOS(舊)', '期貨電子盤類似受眾_IOS', '創意廣告', 'APP_INSTALLS', Decimal('124'), Decimal('6'), None, None, None, None, Decimal('57.94393'), Decimal('0.79439'), Decimal('2140'), '20200101', 'FB'), (Decimal('202001010002'), '籌碼K線_ 競價下載_行銷活動花費無限制', '理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android', '圖片', 'APP_INSTALLS', Decimal('99'), Decimal('9'), None, None, None, None, Decimal('85.05155'), Decimal('1.89003'), Decimal(

""


In [14]:
def Get_SN_RD(TableName):
    get_both = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(get_both.format(TableName, TableName))
    DBdata_cr.execute(get_both.format(TableName, TableName))
    # 獲取最近的日期(p)和流水號(Both_SN_RD)
    Last_SN, Last_RD = DBdata_cr.fetchall()[0]
    print(Last_SN, Last_RD, end = '\n\n')
    
    df = pd.DataFrame(DBdata_cr.fetchall())
    print(DBdata_cr.description, end = '\n\n')
#     df.columns = [ x for x in DBdata_cr.description ]
    conn_DBdata.commit()
    return Last_SN, Last_RD, df

Last_SN, Last_RD, df = Get_SN_RD(TableName)
# print(a[0][1])
# print(a[0][0])
df


        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[FB_AdsData])
        
2021032800053028 20210328

(('Serial_Number', <class 'decimal.Decimal'>, None, 18, 18, 0, False), ('Reporting_Date', <class 'str'>, None, 20, 20, 0, False))



""


In [44]:
# 提取流水號、報告日期
def Get_SN_RD(TableName):
    get_both = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(get_both.format(TableName, TableName))
    DBdata_cr.execute(get_both.format(TableName, TableName))
    Both_SN_RD = DBdata_cr.fetchall()
    conn_DBdata.commit()
    return Both_SN_RD

# 清洗時間(Reporting_time)欄位，並更新
# def Clean_both_SN_RD(uncleaned_data):
#     index = 0
#     list_of_data = [k for k in str(uncleaned_data).strip('[]').strip('()').split(',')]
#     for i in list_of_data:
# #         print(i.strip().strip('Decimal').strip('()').strip("'"), '\n')
#         cleaned_data = i.strip().strip('Decimal').strip('()').strip("'")
        
#         # 將清洗好的資料儲存
#         if index%2 == 0:
#             index += 1
#             try:
#                 cleaned_SN = int(float(cleaned_data))
#             except ValueError as e_reporting_date:
#                 print(e_reporting_date)
            
#         elif index%2 == 1: 
#             index += 1
#             try:
#                 cleaned_RD = int(float(cleaned_data))
#             except ValueError as e_reporting_date:
#                 print(e_reporting_date)
#     return cleaned_SN, cleaned_RD


# 將Reporting_Date回溯1個月前
def RD_date_back(cleaned_RD):
    # 將str變成時間格式(datetime)，才能做計算
    reporting_date = datetime.strptime(str(cleaned_RD), '%Y%m%d')
    
    # 當現在的日期已經over
    if reporting_date.date() < (datetime.now().date() - relativedelta(months = 1)):
        reporting_date = reporting_date.date()
    else:
        # 將日期回溯1個月
        reporting_date = reporting_date.date() - relativedelta(months = 1)
    return reporting_date


# 增加流水號 & 廣告類別
def Expand_df(df, cleaned_SN): 
    # 第幾排插入流水號 
    idx_SN = 0
    idx_CA = 11
    Serial_Number = []
    # 廣告類別
    Category = 'FB'

    # 增加Serial_Number & Catergory(FB, GOOGL, AAPL)
    for order in range(len(df)):
        s_num = str(df.Reporting_Date[order]) + '000' + str(int(str(cleaned_SN)[8:]) + order + 1)
        Serial_Number.append(int(s_num))
    
    # 將資料插入
    df.insert(loc = idx_SN, column = 'Serial_Number', value = Serial_Number)
    df.insert(loc = idx_CA, column = 'Category', value = Category)
    return df


# 將CSV內檔案的
def None_to_Null(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: None if x == '' else x)
    return df 


# 將df的資料備份下來
def Df_to_Csv(df):
    # 將df內的Data儲存成CSV file
    df.to_csv('{}_FBAdsTest.csv'.format(datetime.datetime.now().strftime('%Y%m%d')), index = False, encoding='utf_8_sig')
    return None


# 刪除資料
def DelSQL(TableName, start_time, now_time):
    str_query = """
        DELETE
        FROM [CMAPP].[dbo].[{}]
        WHERE Reporting_Date BETWEEN {} AND {}
        """
    print(str_query.format(TableName, start_time, now_time))
    DBdata_cr.execute(str_query.format(TableName, start_time, now_time))
    conn_DBdata.commit()   


# 寫入資料
def InsertSQL(df, TableName):
    str_query = """
        INSERT INTO [CMAPP].[dbo].[{}]([Serial_Number],[Campaign_Name],[Adset_Name],[Ad_Name],[Objective],[Amount_Spent],[Amount_Install],\
        [Amount_Purchase],[Purchase_Conversion_Value],[Amount_Subscribe],[Subscribe_Conversion_Value],[CPM],[CTR],[Impressions],[Reporting_Date]\
        ,[Category])VALUES({})
        """
    # 將csv DataFrame每筆資料變成list形式
    insert_data = df.values.tolist()
    for i in range(len(insert_data)):
#         print(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))      
        DBdata_cr.execute(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))
        conn_DBdata.commit() 

In [45]:
# 日期計算
def init_time(data_back):
    """
    a_week: 一星期
    a_day: 一天
    end_time: 回推月份 - 1個月
    start_time: 回推月份
    """
    a_week = relativedelta(weeks = 1)
    a_day = relativedelta(days = 1)
    
    # 回推開始月份
    start_time = datetime.now() - data_back*relativedelta(months = 1)
    # 回推開始月份 - 1星期
    end_time = datetime.now() - data_back*relativedelta(months = 1) + relativedelta(weeks = 1)
    return start_time, end_time

## 所有廣告帳號

In [23]:
Act_list = {'元帳號': '132799196821088',
'StockAiPro APP': '783539402272443',
'StockAiPro APP(iOS)': '1143824496065734',
'icheck': '936847127084019',
'icheck(iOS)': '4196385753706403',
'丹尼爾PC+APP': '452504569395856',
'丹尼爾PC+APP(iOS)': '3871892012868660',
'價值K線APP': '795937830999375',
'價值K線APP(iOS)': '294383082255305',
'優惠王-costco同學會': '712103039460113',
'優惠王-costco同學會(iOS)': '467704917976698',
'大大PM': '509426729992360',
'大股東 APP': '753062955394894',
'大股東 APP(iOS)': '1864362753722473',
'我的房價漲多少': '149018927107258',
'我的房價漲多少(iOS)': '454759162500759',
'房屋價值地圖': '157768706197434',
'房屋價值地圖(iOS)': '785822512339192',
'投資小學堂APP': '219944153210593',
'投資小學堂APP(iOS)': '285019616541316',
'新聞爆料同學會': '129225645812364',
'新聞爆料同學會(iOS)': '462416488363153',
'月老': '505050013831773',
'月老(iOS)': '462146505235011',
'期權先生PC+APP': '197791738405867',
'期權先生PC+APP(iOS)': '449681076323070',
'期貨電子盤 APP': '3873072282807287',
'期貨電子盤 APP(iOS)': '170149104947622',
'林恩如PC+APP': '534983857479217',
'林恩如PC+APP(iOS)': '491297538573013',
'權證小哥PC+APP': '1132572753851821',
'權證小哥PC+APP(iOS)': '436996980921253',
'武財神': '428844138551031',
'武財神(iOS)': '298213318597607',
'每日頭條APP': '222102939378962',
'每日頭條APP(iOS)': '551649459143632',
'無聊詹PC+APP': '440198387247896',
'無聊詹PC+APP(iOS)': '1339779816406536',
'租屋雷達': '1086617258501383',
'租屋雷達(iOS)': '1061710770977404',
'算利教官 APP': '480153816358865',
'算利教官 APP(iOS)': '4284601198239102',
'籌碼K線 APP': '372330733913901',
'籌碼K線 APP (iOS)': '136443351648696',
'籌碼K線 PC': '3648034708655701',
'美股夢想家 PC+APP': '279643700237508',
'美股夢想家 PC+APP(iOS)': '478730439945199',
'股人阿勳 APP': '1614448442086305',
'股人阿勳 APP(iOS)': '830300951168633',
'股市Podcast': '448233149813034',
'股市Podcast(iOS)': '1329909827384466',
'股市爆料同學會APP': '238268321010982',
'股市爆料同學會APP(iOS)': '1757870844381494',
'自由人PC+APP': '137512381633886',
'自由人PC+APP(iOS)': '166308061984091',
'艾蜜莉 APP': '319046682694239',
'艾蜜莉 APP(iOS)': '2813564628898257',
'華倫 APP': '476405876831222',
'華倫 APP(iOS)': '426164802018260',
'阿格力 APP': '2767955253458688',
'阿格力 APP(iOS)': '293950655476450',
'阿水PC+APP': '787416052182106',
'阿水PC+APP(iOS)': '278212370485576',
'陳重銘APP': '268527648070343',
'陳重銘APP(iOS)': '458094148869336',
}
print(Act_list.keys())

dict_keys(['元帳號', 'StockAiPro APP', 'StockAiPro APP(iOS)', 'icheck', 'icheck(iOS)', '丹尼爾PC+APP', '丹尼爾PC+APP(iOS)', '價值K線APP', '價值K線APP(iOS)', '優惠王-costco同學會', '優惠王-costco同學會(iOS)', '大大PM', '大股東 APP', '大股東 APP(iOS)', '我的房價漲多少', '我的房價漲多少(iOS)', '房屋價值地圖', '房屋價值地圖(iOS)', '投資小學堂APP', '投資小學堂APP(iOS)', '新聞爆料同學會', '新聞爆料同學會(iOS)', '月老', '月老(iOS)', '期權先生PC+APP', '期權先生PC+APP(iOS)', '期貨電子盤 APP', '期貨電子盤 APP(iOS)', '林恩如PC+APP', '林恩如PC+APP(iOS)', '權證小哥PC+APP', '權證小哥PC+APP(iOS)', '武財神', '武財神(iOS)', '每日頭條APP', '每日頭條APP(iOS)', '無聊詹PC+APP', '無聊詹PC+APP(iOS)', '租屋雷達', '租屋雷達(iOS)', '算利教官 APP', '算利教官 APP(iOS)', '籌碼K線 APP', '籌碼K線 APP (iOS)', '籌碼K線 PC', '美股夢想家 PC+APP', '美股夢想家 PC+APP(iOS)', '股人阿勳 APP', '股人阿勳 APP(iOS)', '股市Podcast', '股市Podcast(iOS)', '股市爆料同學會APP', '股市爆料同學會APP(iOS)', '自由人PC+APP', '自由人PC+APP(iOS)', '艾蜜莉 APP', '艾蜜莉 APP(iOS)', '華倫 APP', '華倫 APP(iOS)', '阿格力 APP', '阿格力 APP(iOS)', '阿水PC+APP', '阿水PC+APP(iOS)', '陳重銘APP', '陳重銘APP(iOS)'])


# 改版of main_prog

In [30]:
# ad_account_id = 'act_{account_id}'.format(account_id = act_id: act_id for key, act_id in Act_list.items())
# print(ad_account_id)
print(range(len(Act_list.items())))
print(act_id for key, act_id in Act_list.items())

range(0, 65)
<generator object <genexpr> at 0x000002351A50FE58>


In [44]:
def main_prog(act_id, start_time, end_time, obj_time):
    # 起始值
    """
    access_token: 廣告金鑰
    ad_account_id: 廣告帳戶
    data_back: 往前爬多久的資料(幾個月)
    fields: FB_Ads行銷廣告資料
    count: 計算整體while迴圈跑幾次
    stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
    """
    access_token = 'EAAJqMzKi2IsBAHrHkHKYzF8Kc1PgskkXweJlxbv2qtNDrIVWdIHPXZBuclZAdFVuCE9tjZAfklwL8JxOmdhcZABjKYJRu9kqXMq3Q2kwBWmdZBjixZCCEkTzMXlrHeW9f3eJoylClpa0FqMUbecNair0MhT74eYZAJtwQxL5aoF4mejBTn2ZBAz6S4FSP6pUy2gZD'
    ad_account_id = 'act_{}'.format(act_id)
    count = 0
    stop_flag = 0
    FacebookAdsApi.init(access_token = access_token)

    # Insight API中的fields資料(控制需要的欄位資料)
    fields = [
        'spend',
        'campaign_name',
        'adset_name',
        'campaign_id',
        'ad_name',
        'actions',
        'objective',
        'action_values',  
        'conversions',
        'conversion_values',
        'impressions',
        'ctr',
        'cpm',
    ]

    # 起始資料放置位置
    Campaign_Name = list()
    Adset_Name = list()
    Ad_Name = list()
    Objective = list()
    Amount_Spent = list()
    Amount_Install = list()
    Amount_Purchase = list()
    Purchase_Conversion_Value = list()
    Amount_Subscribe = list()
    Subscribe_Conversion_Value = list()
    Reporting_Date = list()
    CPM = list()
    CTR = list()
    Impressions = list()


    # 紀錄程式跑多久時間
    run_time = time.time()
   
    # 現在時間
#     now = datetime.now().date()

    # 比較時間前後順序
    while True:
        # 計算迴圈跑幾次
        count += 1
        # Insight API中的fields資料(控制日期條件、資料的層級)
        # start_time和end_time需要為string(%Y-%m-%d)
        params = {
            'time_range': {
                        'since': str(start_time),\
                        'until': str(end_time),
            }, 
            # level從campaign改成adset
            'level': {'ad'},
            'time_increment': '1',
            # 手動設定的歸因: 7天點擊後(和廣告預設值有落差)
            'action_attribution_windows': {'7d_click'},
        }

        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )

        # 了解時間怎麼走
        print(start_time.strftime('%Y-%m-%d'))
        print(end_time.strftime('%Y-%m-%d'))
        
        # stop_flag = 0 代表end_time的時間已經超過現在時間(now_time)
        if stop_flag == 0:
        # 起始時間(start_time)和結束時間(end_time)，一次取一星期
            start_time = end_time + relativedelta(days = 1)
        else:
            start_time = end_time
        
        # 每次都將結束時間(end_time)，往後增加一星期
        end_time = end_time + relativedelta(weeks = 1)

        # 從廣告Inishgts資訊中，清洗、篩選資料
        for acc_insight in acc_insights:     
            campaign_name = ''
            adset_name = ''
            ad_name = ''
            amount_spent = ''
            objective = ''
            amount_subscribe = '' 
            subscribe_conversion_value = ''
            amount_install = '' 
            amount_purchase = ''    
            purchase_conversion_value = ''
            reporting_date = ''          
            cpm = ''
            ctr = ''
            impressions = ''

            # insights/campaign_name得到 "廣告包名稱"
            if 'campaign_name' in acc_insight:
                campaign_name = acc_insight['campaign_name']
                if '⛔️股市爆料同學會' in campaign_name:
                    campaign_name = campaign_name.replace('⛔️', '')

            # insights/adset_name得到 "廣告set名稱"
            if 'adset_name' in acc_insight:
                adset_name = acc_insight['adset_name']
                
            # insights/adset_name得到 "廣告名稱"
            if 'ad_name' in acc_insight:
                ad_name = acc_insight['ad_name']
            
            # insights/cpm得到 "每一次廣告的曝光成本"
            if 'cpm' in acc_insight:
                cpm = acc_insight['cpm']
                
            # insights/ctr得到 "廣告點擊率"
            if 'ctr' in acc_insight:
                ctr = acc_insight['ctr']

            # insights/ctr得到 "廣告觸及人數"
            if 'impressions' in acc_insight:
                impressions = acc_insight['impressions']
            
            # insights/spend得到 "總支出成本"
            if 'spend' in acc_insight:    
                amount_spent = acc_insight['spend']

            # insights/objective得到 "廣告目標"
            if 'objective' in acc_insight:
                objective = acc_insight['objective']

            # insights/date_stop得到 "報表終止日期"
            if 'date_stop' in acc_insight:
                reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

            # insights/actions得到 "軟體下載數/課程購買數"
            if 'actions' in acc_insight:
                for action in acc_insight['actions']:
                    if action['action_type'] == 'mobile_app_install': 
                        amount_install = action['value']
                    if action['action_type'] == 'purchase':
                        amount_purchase = action['value']

            # insights/action_values得到 "購買轉換價值"
            if 'action_values' in acc_insight:
                for ac_values in acc_insight['action_values']:
                    if ac_values['action_type'] == 'omni_purchase':
                        purchase_conversion_value = ac_values['value']

            # insights/conversions得到 "訂閱數"
            if 'conversions' in acc_insight:
                for cv in acc_insight['conversions']:
                    if cv['action_type'] == 'subscribe_total':
                        try:
                            amount_subscribe = cv['7d_click']
                        except:
                            amount_subscribe = cv['value']

            # insights/conversion_values得到 "訂閱轉換價值"             
            if 'conversion_values' in acc_insight:
                for cv_values in acc_insight['conversion_values']:
                    if cv_values['action_type'] == 'subscribe_total':
                        try:
                            subscribe_conversion_value = cv_values['7d_click']
                        except:
                            subscribe_conversion_value = cv_values['value']

            # 將資料以List的形式儲存
            try:
                Campaign_Name.append(campaign_name)
                Adset_Name.append(adset_name)
                Ad_Name.append(ad_name)
                Objective.append(objective)
                Amount_Spent.append(amount_spent)
                Amount_Install.append(amount_install)
                Amount_Purchase.append(amount_purchase)
                Purchase_Conversion_Value.append(purchase_conversion_value)
                Amount_Subscribe.append(amount_subscribe)
                Subscribe_Conversion_Value.append(subscribe_conversion_value)
                Reporting_Date.append(reporting_date)
                CPM.append(cpm)
                CTR.append(ctr)
                Impressions.append(impressions)
            except UnicodeEncodeError as e:
                print(e)
            
            print(acc_insight['campaign_name'], '\n', acc_insight['adset_name'], '\n', acc_insight['ad_name'], '\n', acc_insight)
            

            # 流程時間判斷
#             if now < end_time:
#                 end_time = now
#                 stop_flag += 1       
                
            if obj_time < end_time:
                end_time = obj_time
                stop_flag += 1        


        print('第{}次'.format(count)) 
#         time.sleep(60)
        
        # 當成是執行4次時，暫停時間300秒
        if count%4 == 0:
            time.sleep(60)
        
        # 當開始時間和結束時間相等時，立即結束程式
        if start_time == end_time:
            break
        # 將資料以Dataaframe的方式儲存
        raw_data = {'Campaign_Name': Campaign_Name,
                 'Adset_Name': Adset_Name,
                 'Ad_Name': Ad_Name,
                 'Objective': Objective,
                 'Amount_Spent': Amount_Spent,
                 'Amount_Install': Amount_Install,
                 'Amount_Purchase': Amount_Purchase,
                 'Purchase_Conversion_Value': Purchase_Conversion_Value,
                 'Amount_Subscribe': Amount_Subscribe,
                 'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
                 'CPM': CPM,
                 'CTR': CTR,
                 'Impressions': Impressions,
                 'Reporting_Date': Reporting_Date,}
        df = pd.DataFrame.from_dict(raw_data)

    # flag確立program完整結束
    print('--- %s seconds ---' % (time.time() - run_time))
    
    return df

In [46]:
# 原版的 main_prog
def main_prog(start_time, end_time, obj_time):
    # 起始值
    """
    access_token: 廣告金鑰
    ad_account_id: 廣告帳戶
    data_back: 往前爬多久的資料(幾個月)
    fields: FB_Ads行銷廣告資料
    count: 計算整體while迴圈跑幾次
    stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
    """
    access_token = 'EAAJqMzKi2IsBAHrHkHKYzF8Kc1PgskkXweJlxbv2qtNDrIVWdIHPXZBuclZAdFVuCE9tjZAfklwL8JxOmdhcZABjKYJRu9kqXMq3Q2kwBWmdZBjixZCCEkTzMXlrHeW9f3eJoylClpa0FqMUbecNair0MhT74eYZAJtwQxL5aoF4mejBTn2ZBAz6S4FSP6pUy2gZD'
    ad_account_id = 'act_132799196821088'
    count = 0
    stop_flag = 0
    FacebookAdsApi.init(access_token = access_token)

    # Insight API中的fields資料(控制需要的欄位資料)
    fields = [
        'spend',
        'campaign_name',
        'adset_name',
        'campaign_id',
        'ad_name',
        'actions',
        'objective',
        'action_values',  
        'conversions',
        'conversion_values',
        'impressions',
        'ctr',
        'cpm',
    ]

    # 起始資料放置位置
    Campaign_Name = list()
    Adset_Name = list()
    Ad_Name = list()
    Objective = list()
    Amount_Spent = list()
    Amount_Install = list()
    Amount_Purchase = list()
    Purchase_Conversion_Value = list()
    Amount_Subscribe = list()
    Subscribe_Conversion_Value = list()
    Reporting_Date = list()
    CPM = list()
    CTR = list()
    Impressions = list()


    # 紀錄程式跑多久時間
    run_time = time.time()
   
    # 起始時間、結束時間(For Test)
#     start_time = datetime.now() - relativedelta(months = 5)
#     end_time = start_time + relativedelta(months = 1)
    
    # 起始時間、結束時間
#     start_time, end_time = start_time, end_time
#     start_time, end_time = init_time(data_back = 13)
    
    # 現在時間
    now = datetime.now().date()

    # 比較時間前後順序
    while True:
        # 計算迴圈跑幾次
        count += 1
        # Insight API中的fields資料(控制日期條件、資料的層級)
        # start_time和end_time需要為string(%Y-%m-%d)
        params = {
            'time_range': {
                        'since': str(start_time),\
                        'until': str(end_time),
#                         'since': start_time.strftime('%Y-%m-%d'),\
#                         'until': end_time.strftime('%Y-%m-%d'), 
            }, 
            # level從campaign改成adset
            'level': {'ad'},
            'time_increment': '1',
            # 手動設定的歸因: 7天點擊後(和廣告預設值有落差)
            'action_attribution_windows': {'7d_click'},
        }

        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )

        # 了解時間怎麼走
#         print(start_time.date().strftime('%Y-%m-%d'))
#         print(end_time.date().strftime('%Y-%m-%d'))
        print(start_time.strftime('%Y-%m-%d'))
        print(end_time.strftime('%Y-%m-%d'))
        
        # stop_flag = 0 代表end_time的時間已經超過現在時間(now_time)
        if stop_flag == 0:
        # 起始時間(start_time)和結束時間(end_time)，一次取一星期
            start_time = end_time + relativedelta(days = 1)
        else:
            start_time = end_time
        
        # 每次都將結束時間(end_time)，往後增加一星期
        end_time = end_time + relativedelta(weeks = 1)

        # 從廣告Inishgts資訊中，清洗、篩選資料
        for acc_insight in acc_insights:     
            campaign_name = ''
            adset_name = ''
            ad_name = ''
            amount_spent = ''
            objective = ''
            amount_subscribe = '' 
            subscribe_conversion_value = ''
            amount_install = '' 
            amount_purchase = ''    
            purchase_conversion_value = ''
            reporting_date = ''          
            cpm = ''
            ctr = ''
            impressions = ''

            # insights/campaign_name得到 "廣告包名稱"
            if 'campaign_name' in acc_insight:
                campaign_name = acc_insight['campaign_name']
                if '⛔️股市爆料同學會' in campaign_name:
                    campaign_name = campaign_name.replace('⛔️', '')

            # insights/adset_name得到 "廣告set名稱"
            if 'adset_name' in acc_insight:
                adset_name = acc_insight['adset_name']
                
            # insights/adset_name得到 "廣告名稱"
            if 'ad_name' in acc_insight:
                ad_name = acc_insight['ad_name']
            
            # insights/cpm得到 "每一次廣告的曝光成本"
            if 'cpm' in acc_insight:
                cpm = acc_insight['cpm']
                
            # insights/ctr得到 "廣告點擊率"
            if 'ctr' in acc_insight:
                ctr = acc_insight['ctr']

            # insights/ctr得到 "廣告觸及人數"
            if 'impressions' in acc_insight:
                impressions = acc_insight['impressions']
            
            # insights/spend得到 "總支出成本"
            if 'spend' in acc_insight:    
                amount_spent = acc_insight['spend']

            # insights/objective得到 "廣告目標"
            if 'objective' in acc_insight:
                objective = acc_insight['objective']

            # insights/date_stop得到 "報表終止日期"
            if 'date_stop' in acc_insight:
                reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

            # insights/actions得到 "軟體下載數/課程購買數"
            if 'actions' in acc_insight:
                for action in acc_insight['actions']:
                    if action['action_type'] == 'mobile_app_install': 
                        amount_install = action['value']
                    if action['action_type'] == 'purchase':
                        amount_purchase = action['value']

            # insights/action_values得到 "購買轉換價值"
            if 'action_values' in acc_insight:
                for ac_values in acc_insight['action_values']:
                    if ac_values['action_type'] == 'omni_purchase':
                        purchase_conversion_value = ac_values['value']

            # insights/conversions得到 "訂閱數"
            if 'conversions' in acc_insight:
                for cv in acc_insight['conversions']:
                    if cv['action_type'] == 'subscribe_total':
                        try:
                            amount_subscribe = cv['7d_click']
                        except:
                            amount_subscribe = cv['value']

            # insights/conversion_values得到 "訂閱轉換價值"             
            if 'conversion_values' in acc_insight:
                for cv_values in acc_insight['conversion_values']:
                    if cv_values['action_type'] == 'subscribe_total':
                        try:
                            subscribe_conversion_value = cv_values['7d_click']
                        except:
                            subscribe_conversion_value = cv_values['value']

            # 將資料以List的形式儲存
            try:
                Campaign_Name.append(campaign_name)
                Adset_Name.append(adset_name)
                Ad_Name.append(ad_name)
                Objective.append(objective)
                Amount_Spent.append(amount_spent)
                Amount_Install.append(amount_install)
                Amount_Purchase.append(amount_purchase)
                Purchase_Conversion_Value.append(purchase_conversion_value)
                Amount_Subscribe.append(amount_subscribe)
                Subscribe_Conversion_Value.append(subscribe_conversion_value)
                Reporting_Date.append(reporting_date)
                CPM.append(cpm)
                CTR.append(ctr)
                Impressions.append(impressions)
            except UnicodeEncodeError as e:
                print(e)
            
#             print(acc_insight['campaign_name'], '\n', acc_insight['adset_name'], '\n', acc_insight['ad_name'], '\n', acc_insight)
            
            # 如果結束時間大於現在時間，則立即停止迴圈
#             if now < end_time.date():
#                 end_time = now
#                 stop_flag += 1        
#             elif obj_time.date() < end_time.date():
#                 break

            if now < end_time:
                end_time = now
                stop_flag += 1       
                
            elif obj_time < end_time:
                end_time = obj_time
                stop_flag += 1        


        print('第{}次'.format(count)) 
#         time.sleep(60)
        
        # 當成是執行4次時，暫停時間300秒
        if count%4 == 0:
            time.sleep(60)
        
        # 當開始時間和結束時間相等時，立即結束程式
        if start_time == end_time:
            break
        # 將資料以Dataaframe的方式儲存
        raw_data = {'Campaign_Name': Campaign_Name,
                 'Adset_Name': Adset_Name,
                 'Ad_Name': Ad_Name,
                 'Objective': Objective,
                 'Amount_Spent': Amount_Spent,
                 'Amount_Install': Amount_Install,
                 'Amount_Purchase': Amount_Purchase,
                 'Purchase_Conversion_Value': Purchase_Conversion_Value,
                 'Amount_Subscribe': Amount_Subscribe,
                 'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
                 'CPM': CPM,
                 'CTR': CTR,
                 'Impressions': Impressions,
                 'Reporting_Date': Reporting_Date,}
        df = pd.DataFrame.from_dict(raw_data)

    # flag確立program完整結束
    print('--- %s seconds ---' % (time.time() - run_time))
    
    return df

In [46]:
for key, act_id in Act_list.items():
    # 起始時間
    start_time = datetime(2021, 1, 1).date()
    # 每次爬蟲一段時間
    end_time = start_time + relativedelta(weeks = 1)
    # 爬蟲終止時間
    obj_time =  datetime(2021, 1, 10).date()
    df = main_prog('2813564628898257', start_time, end_time, obj_time)

2021-01-01
2021-01-08
第1次
2021-01-09
2021-01-15
第2次
2021-01-16
2021-01-22
第3次
2021-01-23
2021-01-29
第4次


KeyboardInterrupt: 

In [41]:
for key, act_id in Act_list.items():
    ad_account_id = 'act_{}'.format(act_id)
    print(ad_account_id)

act_132799196821088
act_783539402272443
act_1143824496065734
act_936847127084019
act_4196385753706403
act_452504569395856
act_3871892012868660
act_795937830999375
act_294383082255305
act_712103039460113
act_467704917976698
act_509426729992360
act_753062955394894
act_1864362753722473
act_149018927107258
act_454759162500759
act_157768706197434
act_785822512339192
act_219944153210593
act_285019616541316
act_129225645812364
act_462416488363153
act_505050013831773
act_462146505235011
act_197791738405867
act_449681076323070
act_3873072282807287
act_170149104947622
act_534983857479217
act_491297538573013
act_1132572753851821
act_436996980921253
act_428844138551031
act_298213318597607
act_222102939378962
act_551649459143632
act_440198387247896
act_1339779816406536
act_1086617258501383
act_1061710770977404
act_480153816358865
act_4284601198239102
act_372330733913901
act_136443351648696
act_3648034708655701
act_279643700237508
act_478730439945199
act_1614448442086305
act_830300951168633
act_4482

In [42]:
# start_time, end_time = init_time(13)
# start_time = datetime(2020, 1, 1).date()
# end_time = start_time + relativedelta(weeks = 1)
# obj_time =  datetime(2020, 1, 10).date()

for i in range(len(Act_list)):
    try: 
        for key, act_id in Act_list.items():
            start_time = datetime(2021, 1, 1).date()
            end_time = start_time + relativedelta(weeks = 1)
            obj_time =  datetime(2021, 1, 10).date()
            df = main_prog(act_id, start_time, end_time, obj_time)
#             print(df)
        sucMsg = 'Working well'
    except Exception as e:
        error_class = e.__class__.__name__ #取得錯誤類型
        detail = e.args[0] #取得詳細內容
        _, _, tb = sys.exc_info() #取得Call Stack
        lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
        fileName = lastCallStack[0] #取得發生的檔案名稱
        lineNum = lastCallStack[1] #取得發生的行號
        funcName = lastCallStack[2] #取得發生的函數名稱
        errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
        print(errMsg, '\n')

2021-01-01
2021-01-08
第1次
2021-01-09
2021-01-10
第2次
--- 29.75029444694519 seconds ---
                          Campaign_Name                           Adset_Name  \
0            武財神_APP下載_Android_20200723       武財神__Android_25-65+_興趣_道教_自動版位   
1            武財神_APP下載_Android_20200723  武財神__Android_25-65+_類似受眾_安裝APP_自動版位   
2                武財神_APP下載_iOS_20200723           武財神__iOS_25-65+_興趣_道教_自動版位   
3                武財神_APP下載_iOS_20200723      武財神__iOS_25-65+_類似受眾_安裝APP_自動版位   
4                武財神_APP下載_iOS_20200723         武財神__iOS_25-65+_類似受眾_上香_自動版位   
5             20200922_艾蜜莉APP_IOS_下載_軟體                     財報投資受眾_IOS (便宜價)   
6         20200922_艾蜜莉APP_Android_下載_軟體                 財報投資受眾_Android (便宜價)   
7             20200922_艾蜜莉APP_IOS_下載_軟體                    財報投資受眾_IOS (股災影片)   
8         20200922_艾蜜莉APP_Android_下載_軟體                財報投資受眾_Android (股災影片)   
9         20200922_艾蜜莉APP_Android_下載_軟體                 基本面受眾_Android (股災影片)   
10        20200922_艾蜜莉APP_Android_

KeyboardInterrupt: 

In [37]:
df

,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date
0,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,139,7,1,8888,,,57.178116,1.192925,2431,20200101
1,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,124,6,,,,,57.943925,0.794393,2140,20200101
2,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android,圖片,APP_INSTALLS,99,9,,,,,85.051546,1.890034,1164,20200101
3,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限50元_圖片_App畫面_android,籌碼K線_前5%類似受眾_20180927,APP_INSTALLS,93,3,,,,,90.291262,2.135922,1030,20200101
4,期貨電子盤_下載_IOS(舊),安裝過籌碼K線APP_IOS,期貨電子盤,APP_INSTALLS,1,,,,,,50,5,20,20200101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,武財神_點燈活動,最近活躍且有購買的類似受眾,前5%活躍類似受眾,CONVERSIONS,1148,,6,8966,,,85.213777,2.056116,13472,20200122
670,武財神_點燈活動,曾瀏覽武財神LP的人,曾瀏覽武財神LP的人,CONVERSIONS,579,,3,2997,,,69.109573,3.592743,8378,20200122
671,武財神_點燈活動,對理財寶粉專財運文章有興趣的人,對理財寶粉專財運文章有興趣的人,CONVERSIONS,867,,1,999,,,160.793769,3.449555,5392,20200122
672,武財神_點燈活動,與理財寶互動的人,與理財寶互動的人,CONVERSIONS,587,,3,11589,,,63.363558,3.238342,9264,20200122


In [48]:
# 插入新的欄位
import pandas as pd 
import datetime
import numpy as np

# 第幾排插入流水號 & 廣告類別
idx_SN = 0
idx_CA = 15
Serial_Number = []
# 廣告類別
Category = 'FB'

# 增加Serial_Number & Catergory(FB, GOOGL, AAPL)
for order in range(len(df)):
    s_num = str(df['Reporting_Date'][order]) + '000' + str(order)
    Serial_Number.append(int(s_num))

df.insert(loc = idx_SN, column = 'Serial_Number', value = Serial_Number)
df.insert(loc = idx_CA, column = 'Category', value = Category)
df

,Serial_Number,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Category
0,202001010000,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,139,7,1,8888,,,57.178116,1.192925,2431,20200101,FB
1,202001010001,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,124,6,,,,,57.943925,0.794393,2140,20200101,FB
2,202001010002,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android,圖片,APP_INSTALLS,99,9,,,,,85.051546,1.890034,1164,20200101,FB
3,202001010003,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限50元_圖片_App畫面_android,籌碼K線_前5%類似受眾_20180927,APP_INSTALLS,93,3,,,,,90.291262,2.135922,1030,20200101,FB
4,202001010004,期貨電子盤_下載_IOS(舊),安裝過籌碼K線APP_IOS,期貨電子盤,APP_INSTALLS,1,,,,,,50,5,20,20200101,FB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,20200122000669,武財神_點燈活動,最近活躍且有購買的類似受眾,前5%活躍類似受眾,CONVERSIONS,1148,,6,8966,,,85.213777,2.056116,13472,20200122,FB
670,20200122000670,武財神_點燈活動,曾瀏覽武財神LP的人,曾瀏覽武財神LP的人,CONVERSIONS,579,,3,2997,,,69.109573,3.592743,8378,20200122,FB
671,20200122000671,武財神_點燈活動,對理財寶粉專財運文章有興趣的人,對理財寶粉專財運文章有興趣的人,CONVERSIONS,867,,1,999,,,160.793769,3.449555,5392,20200122,FB
672,20200122000672,武財神_點燈活動,與理財寶互動的人,與理財寶互動的人,CONVERSIONS,587,,3,11589,,,63.363558,3.238342,9264,20200122,FB


In [ ]:
# 將df擴增流水號和廣告種類標籤
df = Expand_df(df, cleaned_SN)
df

In [49]:
# 將df內的空值""，儲存成None的型態
df = None_to_Null(df)
df

,Serial_Number,Campaign_Name,Adset_Name,Ad_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,CPM,CTR,Impressions,Reporting_Date,Category
0,202001010000,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,創意廣告,APP_INSTALLS,139,7,1,8888,None,None,57.178116,1.192925,2431,20200101,FB
1,202001010001,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,創意廣告,APP_INSTALLS,124,6,None,None,None,None,57.943925,0.794393,2140,20200101,FB
2,202001010002,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android,圖片,APP_INSTALLS,99,9,None,None,None,None,85.051546,1.890034,1164,20200101,FB
3,202001010003,籌碼K線_ 競價下載_行銷活動花費無限制,理財寶相關受眾_出價上限50元_圖片_App畫面_android,籌碼K線_前5%類似受眾_20180927,APP_INSTALLS,93,3,None,None,None,None,90.291262,2.135922,1030,20200101,FB
4,202001010004,期貨電子盤_下載_IOS(舊),安裝過籌碼K線APP_IOS,期貨電子盤,APP_INSTALLS,1,None,None,None,None,None,50,5,20,20200101,FB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,20200122000669,武財神_點燈活動,最近活躍且有購買的類似受眾,前5%活躍類似受眾,CONVERSIONS,1148,None,6,8966,None,None,85.213777,2.056116,13472,20200122,FB
670,20200122000670,武財神_點燈活動,曾瀏覽武財神LP的人,曾瀏覽武財神LP的人,CONVERSIONS,579,None,3,2997,None,None,69.109573,3.592743,8378,20200122,FB
671,20200122000671,武財神_點燈活動,對理財寶粉專財運文章有興趣的人,對理財寶粉專財運文章有興趣的人,CONVERSIONS,867,None,1,999,None,None,160.793769,3.449555,5392,20200122,FB
672,20200122000672,武財神_點燈活動,與理財寶互動的人,與理財寶互動的人,CONVERSIONS,587,None,3,11589,None,None,63.363558,3.238342,9264,20200122,FB


In [50]:
# 將df儲存成csv檔愛保存
Df_to_Csv(df)

In [51]:
# 將RD儲存成str型態，將df資料丟入資料庫內
df['Reporting_Date'] = df['Reporting_Date'].astype('str') #將資料型態轉為字串,但實際上不轉換寫入SQL時,SQL也會自動處理
InsertSQL(df, TableName)

In [18]:
a = datetime(2020, 4, 25).date()
print(a)


2020-04-25


In [23]:
start_time = datetime(2020, 1, 1).date()
end_time = start_time + relativedelta(weeks = 1)
print(end_time)
obj_time =  datetime(2020, 2, 1).date()
print(obj_time)

2020-01-08
2020-02-01


In [14]:
a = datetime.datetime.now().strftime('%Y%m%d')
print(a)

20210127


## 20210205 V-1 找尋[1](https://www.cedwards.info/projects/facebookeda/#chrisbow)內的Total Conversion和Approved Conversion 

In [ ]:
# 透過連接FB API來達到特定資料獲取，
# 並

def main_prog(start_time, end_time, obj_time):
    # 起始值
    """
    access_token: 廣告金鑰
    ad_account_id: 廣告帳戶
    data_back: 往前爬多久的資料(幾個月)
    fields: FB_Ads行銷廣告資料
    count: 計算整體while迴圈跑幾次
    stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
    """
    access_token = 'EAAU5W2nIXuUBAIjps8JaNyhVzjh5mCywvWJDW6tleQXMGEl2xJMdlrfWDZB4wxZCRUU0Ed0FAKXT2acm31ZATt39m3u62ash9SP1La5BHgQOtEIBhCa5prRVDOuzZCPHAWeGPjBWCeZCegS3cmsu8ZBsfvURWFibQByAgGVqC53OHkxZA0pxixoCaXp1fKkLQgZD'
    ad_account_id = 'act_132799196821088'
    count = 0
    stop_flag = 0
    FacebookAdsApi.init(access_token = access_token)

    # Insight API中的fields資料(控制需要的欄位資料)
    fields = [
        'spend',
        'campaign_name',
        'adset_name',
        'campaign_id',
        'ad_name',
        'actions',
        'objective',
        'action_values',  
        'conversions',
        'conversion_values',
        'impressions',
        'ctr',
        'cpm',
    ]

    # 起始資料放置位置
    Campaign_Name = list()
    Adset_Name = list()
    Ad_Name = list()
    Objective = list()
    Amount_Spent = list()
    Amount_Install = list()
    Amount_Purchase = list()
    Purchase_Conversion_Value = list()
    Amount_Subscribe = list()
    Subscribe_Conversion_Value = list()
    Reporting_Date = list()
    CPM = list()
    CTR = list()
    Impressions = list()


    # 紀錄程式跑多久時間
    run_time = time.time()
   
    # 起始時間、結束時間(For Test)
#     start_time = datetime.now() - relativedelta(months = 5)
#     end_time = start_time + relativedelta(months = 1)
    
    # 起始時間、結束時間
#     start_time, end_time = start_time, end_time
#     start_time, end_time = init_time(data_back = 13)
    
    # 現在時間
    now = datetime.now().date()

    # 比較時間前後順序
    while True:
        # 計算迴圈跑幾次
        count += 1
        # Insight API中的fields資料(控制日期條件、資料的層級)
        # start_time和end_time需要為string(%Y-%m-%d)
        params = {
            'time_range': {
                        'since': str(start_time),\
                        'until': str(end_time),
#                         'since': start_time.strftime('%Y-%m-%d'),\
#                         'until': end_time.strftime('%Y-%m-%d'), 
            }, 
            # level從campaign改成adset
            'level': {'ad'},
            'time_increment': '1',
            # 手動設定的歸因: 7天點擊後(和廣告預設值有落差)
            'action_attribution_windows': {'7d_click'},
        }

        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )

        # 了解時間怎麼走
#         print(start_time.date().strftime('%Y-%m-%d'))
#         print(end_time.date().strftime('%Y-%m-%d'))
        print(start_time.strftime('%Y-%m-%d'))
        print(end_time.strftime('%Y-%m-%d'))
        
        # stop_flag = 0 代表end_time的時間已經超過現在時間(now_time)
        if stop_flag == 0:
        # 起始時間(start_time)和結束時間(end_time)，一次取一星期
            start_time = end_time + relativedelta(days = 1)
        else:
            start_time = end_time
        
        # 每次都將結束時間(end_time)，往後增加一星期
        end_time = end_time + relativedelta(weeks = 1)

        # 從廣告Inishgts資訊中，清洗、篩選資料
        for acc_insight in acc_insights:     
            campaign_name = ''
            adset_name = ''
            ad_name = ''
            amount_spent = ''
            objective = ''
            amount_subscribe = '' 
            subscribe_conversion_value = ''
            amount_install = '' 
            amount_purchase = ''    
            purchase_conversion_value = ''
            reporting_date = ''          
            cpm = ''
            ctr = ''
            impressions = ''

            # insights/campaign_name得到 "廣告包名稱"
            if 'campaign_name' in acc_insight:
                campaign_name = acc_insight['campaign_name']
                if '⛔️股市爆料同學會' in campaign_name:
                    campaign_name = campaign_name.replace('⛔️', '')

            # insights/adset_name得到 "廣告set名稱"
            if 'adset_name' in acc_insight:
                adset_name = acc_insight['adset_name']
                
            # insights/adset_name得到 "廣告名稱"
            if 'ad_name' in acc_insight:
                ad_name = acc_insight['ad_name']
            
            # insights/cpm得到 "每一次廣告的曝光成本"
            if 'cpm' in acc_insight:
                cpm = acc_insight['cpm']
                
            # insights/ctr得到 "廣告點擊率"
            if 'ctr' in acc_insight:
                ctr = acc_insight['ctr']

            # insights/ctr得到 "廣告觸及人數"
            if 'impressions' in acc_insight:
                impressions = acc_insight['impressions']
            
            # insights/spend得到 "總支出成本"
            if 'spend' in acc_insight:    
                amount_spent = acc_insight['spend']

            # insights/objective得到 "廣告目標"
            if 'objective' in acc_insight:
                objective = acc_insight['objective']

            # insights/date_stop得到 "報表終止日期"
            if 'date_stop' in acc_insight:
                reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

            # insights/actions得到 "軟體下載數/課程購買數"
            if 'actions' in acc_insight:
                for action in acc_insight['actions']:
                    if action['action_type'] == 'mobile_app_install': 
                        amount_install = action['value']
                    if action['action_type'] == 'purchase':
                        amount_purchase = action['value']

            # insights/action_values得到 "購買轉換價值"
            if 'action_values' in acc_insight:
                for ac_values in acc_insight['action_values']:
                    if ac_values['action_type'] == 'omni_purchase':
                        purchase_conversion_value = ac_values['value']

            # insights/conversions得到 "訂閱數"
            if 'conversions' in acc_insight:
                for cv in acc_insight['conversions']:
                    if cv['action_type'] == 'subscribe_total':
                        try:
                            amount_subscribe = cv['7d_click']
                        except:
                            amount_subscribe = cv['value']

            # insights/conversion_values得到 "訂閱轉換價值"             
            if 'conversion_values' in acc_insight:
                for cv_values in acc_insight['conversion_values']:
                    if cv_values['action_type'] == 'subscribe_total':
                        try:
                            subscribe_conversion_value = cv_values['7d_click']
                        except:
                            subscribe_conversion_value = cv_values['value']

            # 將資料以List的形式儲存
            try:
                Campaign_Name.append(campaign_name)
                Adset_Name.append(adset_name)
                Ad_Name.append(ad_name)
                Objective.append(objective)
                Amount_Spent.append(amount_spent)
                Amount_Install.append(amount_install)
                Amount_Purchase.append(amount_purchase)
                Purchase_Conversion_Value.append(purchase_conversion_value)
                Amount_Subscribe.append(amount_subscribe)
                Subscribe_Conversion_Value.append(subscribe_conversion_value)
                Reporting_Date.append(reporting_date)
                CPM.append(cpm)
                CTR.append(ctr)
                Impressions.append(impressions)
            except UnicodeEncodeError as e:
                print(e)
            
#             print(acc_insight['campaign_name'], '\n', acc_insight['adset_name'], '\n', acc_insight['ad_name'], '\n', acc_insight)
            
            # 如果結束時間大於現在時間，則立即停止迴圈
#             if now < end_time.date():
#                 end_time = now
#                 stop_flag += 1        
#             elif obj_time.date() < end_time.date():
#                 break

            if now < end_time:
                end_time = now
                stop_flag += 1       
                
            elif obj_time < end_time:
                end_time = obj_time
                stop_flag += 1        


        print('第{}次'.format(count)) 
#         time.sleep(60)
        
        # 當成是執行4次時，暫停時間300秒
        if count%4 == 0:
            time.sleep(60)
        
        # 當開始時間和結束時間相等時，立即結束程式
        if start_time == end_time:
            break
        # 將資料以Dataaframe的方式儲存
        raw_data = {'Campaign_Name': Campaign_Name,
                 'Adset_Name': Adset_Name,
                 'Ad_Name': Ad_Name,
                 'Objective': Objective,
                 'Amount_Spent': Amount_Spent,
                 'Amount_Install': Amount_Install,
                 'Amount_Purchase': Amount_Purchase,
                 'Purchase_Conversion_Value': Purchase_Conversion_Value,
                 'Amount_Subscribe': Amount_Subscribe,
                 'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
                 'CPM': CPM,
                 'CTR': CTR,
                 'Impressions': Impressions,
                 'Reporting_Date': Reporting_Date,}
        df = pd.DataFrame.from_dict(raw_data)

    # flag確立program完整結束
    print('--- %s seconds ---' % (time.time() - run_time))
    
    return df